In [ ]:
import pandas as pd
import time, datetime, numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

## Mounting Google Drive to Collab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/CS4248/esnli_train.csv')
test = pd.read_csv('/content/drive/MyDrive/CS4248/esnli_test.csv')

Mounted at /content/drive


In [ ]:
df.shape

(259999, 10)

In [ ]:
df.head()

,pairID,gold_label,Sentence1,Sentence2,Explanation_1,WorkerId,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1
0,3416050480.jpg#4r1n,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,the person is not necessarily training his horse,AF0PI3RISB5Q7,A person on a horse jumps over a broken down a...,A person is *training* *his* *horse* for a co...,{},"3,4,5"
1,3416050480.jpg#4r1c,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",One cannot be on a jumping horse cannot be a d...,A36ZT2WFIA2HMF,A person *on* *a* *horse* *jumps* over a brok...,"A person *is* *at* *a* *diner,* *ordering* an...","4,2,3,5","2,5,4,3,6"
2,3416050480.jpg#4r1e,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",a broken down airplane is outdoors,A2GK75ZQTX2RDZ,A person on a horse jumps over *a* *broken* *...,"A person is *outdoors,* on a horse.","8,9,10,7",3
3,2267923837.jpg#2r1n,neutral,Children smiling and waving at camera,They are smiling at their parents,Just because they are smiling and waving at a ...,A18TOIDG32QICP,Children smiling and waving at camera,They are smiling *at* *their* *parents*,{},"5,3,4"
4,2267923837.jpg#2r1e,entailment,Children smiling and waving at camera,There are children present,The children must be present to see them smili...,AEX0YE6TUZRHT,*Children* *smiling* *and* *waving* at camera,There are children *present*,"0,1,3,2",3


## Utility Functions

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def select_cols(df, col_list):
    '''
    Select columns from a dataframe
    '''
    return df[col_list]

## Data Pre-processing Utility Functions

In [ ]:
def combine_sentences(df, col_list):

    # start_token = '[CLS]'
    # seperator = '[SEP]'

    # combined_text_list = [f'{start_token}{s1}{seperator}{s2}' for s1, s2 in zip(sentence1, sentence2)]
    # return combined_text_list

    results_df = df.copy()

    results_df['combined_text'] = '[CLS]' + results_df[col_list].astype(str).agg('[SEP]'.join, axis=1)

    return results_df

def permute_words(explanation):
    words = explanation.split()
    random.shuffle(words)
    return ' '.join(words)

Train/Test Input Data Handling

In [ ]:
# target_cols = ['Sentence1', 'Sentence2', 'gold_label'] # Premise, Hypothesis
# target_cols = ['Sentence1', 'Sentence2', 'Explanation_1', 'gold_label'] # Premise, Hypothesis, Explanation
# target_cols = ['Sentence2', 'Sentence1', 'gold_label'] # Hypothesis, Premise
target_cols = ['Sentence1', 'Explanation_1', 'Sentence2', 'gold_label'] # Premise, Explanation, Hypothesis

df = select_cols(df, target_cols)
test_df = select_cols(test, target_cols)

df.head()

,Sentence1,Explanation_1,Sentence2,gold_label
0,A person on a horse jumps over a broken down a...,the person is not necessarily training his horse,A person is training his horse for a competition.,neutral
1,A person on a horse jumps over a broken down a...,One cannot be on a jumping horse cannot be a d...,"A person is at a diner, ordering an omelette.",contradiction
2,A person on a horse jumps over a broken down a...,a broken down airplane is outdoors,"A person is outdoors, on a horse.",entailment
3,Children smiling and waving at camera,Just because they are smiling and waving at a ...,They are smiling at their parents,neutral
4,Children smiling and waving at camera,The children must be present to see them smili...,There are children present,entailment


In [ ]:
df['Explanation_1'] = df['Explanation_1'].astype(str).apply(permute_words)
test_df['Explanation_1'] = test_df['Explanation_1'].astype(str).apply(permute_words)


df.head()

<ipython-input-9-30ef9b3dcd30>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Explanation_1'] = test_df['Explanation_1'].astype(str).apply(permute_words)


,Sentence1,Explanation_1,Sentence2,gold_label
0,A person on a horse jumps over a broken down a...,not is horse the training necessarily his person,A person is training his horse for a competition.,neutral
1,A person on a horse jumps over a broken down a...,cannot be jumping diner ordering a food. One b...,"A person is at a diner, ordering an omelette.",contradiction
2,A person on a horse jumps over a broken down a...,a down outdoors is airplane broken,"A person is outdoors, on a horse.",entailment
3,Children smiling and waving at camera,or and does anyone at anyone it not smiling a ...,They are smiling at their parents,neutral
4,Children smiling and waving at camera,waving. The them smiling children and to be se...,There are children present,entailment


In [ ]:
df = combine_sentences(df, target_cols[:-1])
test_df = combine_sentences(test_df, target_cols[:-1])

df.head()

lables = {
    'entailment': 0,
    'neutral': 1,
    'contradiction': 2
}

df['labels'] = df['gold_label'].map(lables)
test_df['labels'] = test_df['gold_label'].map(lables)

In [ ]:
df.head()

,Sentence1,Explanation_1,Sentence2,gold_label,combined_text,labels
0,A person on a horse jumps over a broken down a...,not is horse the training necessarily his person,A person is training his horse for a competition.,neutral,[CLS]A person on a horse jumps over a broken d...,1
1,A person on a horse jumps over a broken down a...,cannot be jumping diner ordering a food. One b...,"A person is at a diner, ordering an omelette.",contradiction,[CLS]A person on a horse jumps over a broken d...,2
2,A person on a horse jumps over a broken down a...,a down outdoors is airplane broken,"A person is outdoors, on a horse.",entailment,[CLS]A person on a horse jumps over a broken d...,0
3,Children smiling and waving at camera,or and does anyone at anyone it not smiling a ...,They are smiling at their parents,neutral,[CLS]Children smiling and waving at camera[SEP...,1
4,Children smiling and waving at camera,waving. The them smiling children and to be se...,There are children present,entailment,[CLS]Children smiling and waving at camera[SEP...,0


In [ ]:
df['combined_text'][0]

'[CLS]A person on a horse jumps over a broken down airplane.[SEP]not is horse the training necessarily his person[SEP]A person is training his horse for a competition.'

In [ ]:
X = df['combined_text']
y = df['labels']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((233999,), (26000,), (233999,), (26000,))

In [ ]:
X_test = test_df['combined_text']
y_test = test_df['labels']

X_test.shape, y_test.shape

((9824,), (9824,))

In [ ]:
class NliDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# tokenize train/validation data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True)

# tokenize test data
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# creating Dataset objects
train_dataset = NliDataset(train_encodings, y_train.tolist())
val_dataset = NliDataset(val_encodings, y_val.tolist())
test_dataset = NliDataset(test_encodings, y_test.tolist())

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)

#unfreezing layer 11 and the classifier. note: the pooler is still frozen
for name, param in model.named_parameters():
    if 'classifier' not in name and '11' not in name:
        param.requires_grad = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Using CPU.")

print("Current device model is on:", model.device)

CUDA is available. Using GPU: Tesla V100-SXM2-16GB
Current device model is on: cuda:0


In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5,
                  eps = 1e-8
                 )

epochs = 2
total_steps = len(train_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Begin Training Loop
loss_values = []

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # time taken for each epoch
    t0 = time.time()

    total_loss = 0

    model.train()

    for step, batch in enumerate(train_loader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)

        # clear previously calculated gradient before backward pass
        model.zero_grad()

        # forward pass
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs.loss

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_loss += loss.item()

        # backwar pass
        loss.backward()

        # Clip the norm of the gradients to 1.0, helps prevents "exploding gradient"
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...
  Batch    40  of  14,625.    Elapsed: 0:00:04.
  Batch    80  of  14,625.    Elapsed: 0:00:06.
  Batch   120  of  14,625.    Elapsed: 0:00:09.
  Batch   160  of  14,625.    Elapsed: 0:00:12.
  Batch   200  of  14,625.    Elapsed: 0:00:14.
  Batch   240  of  14,625.    Elapsed: 0:00:17.
  Batch   280  of  14,625.    Elapsed: 0:00:19.
  Batch   320  of  14,625.    Elapsed: 0:00:22.
  Batch   360  of  14,625.    Elapsed: 0:00:25.
  Batch   400  of  14,625.    Elapsed: 0:00:27.
  Batch   440  of  14,625.    Elapsed: 0:00:30.
  Batch   480  of  14,625.    Elapsed: 0:00:32.
  Batch   520  of  14,625.    Elapsed: 0:00:35.
  Batch   560  of  14,625.    Elapsed: 0:00:38.
  Batch   600  of  14,625.    Elapsed: 0:00:40.
  Batch   640  of  14,625.    Elapsed: 0:00:43.
  Batch   680  of  14,625.    Elapsed: 0:00:45.
  Batch   720  of  14,625.    Elapsed: 0:00:48.
  Batch   760  of  14,625.    Elapsed: 0:00:51.
  Batch   800  of  14,625.    Elapsed: 0:00:5

In [ ]:
model.eval()

predictions, true_labels = [], []

# load test cases into GPU in batches
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = batch['labels'].to('cpu').numpy()

    predictions.append(logits)
    true_labels.append(label_ids)

predictions = np.argmax(np.concatenate(predictions, axis=0), axis=1)
true_labels = np.concatenate(true_labels, axis=0)

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)

precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
print(f"Precision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Accuracy: 0.9452361563517915
Precision: 0.94519695511975
Recall: 0.9452361563517915
F1 Score: 0.9451319628607912


In [ ]:
model_save_path = "/content/drive/MyDrive/CS4248/model_roberta.pth"
optimizer_save_path = "/content/drive/MyDrive/CS4248/model_roberta_optimizer.pth"

# Save the model, optimizer and encodings state
torch.save(model.state_dict(), model_save_path)
torch.save(optimizer.state_dict(), optimizer_save_path)